REMOVE THIS MARKDOWN BEFORE SUBMISSION:

nothing here fuckign works on colab, u will need to download this and run everything either on linux or on wsl.

ull need tensorflow and magents, then switch to the magenta env, set your kernel to magenta also;-;

all pkgs i have: (accurate as of 4:33pm 2023 03 25):

Setup Environment

In [1]:
# @title Setup Enviroment
import warnings
print('Importing libraries')
from absl import app
from absl import logging
import pickle
from magenta.models.music_vae import TrainedModel
from magenta.models.music_vae import configs
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import config
import apache_beam as beam
from apache_beam.metrics import Metrics
import note_seq
import song_utils
print('Set globals and helpers')
MINI_DATASET = True #set to False for full dataset, True for smaller dataset
print('Done')

Importing libraries


/home/david/.local/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/david/.local/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


Set globals and helpers
Done


Define function to process each Note Sequence (NS) into a Latent Embedding (LE)

In [2]:
# @title Encode Class
class Encode(beam.DoFn):
    def setup(self):
        logging.info('Loading pre-trained model')
        self.model_config = config.MUSIC_VAE_CONFIG['melody-2-big']
        self.model = TrainedModel(self.model_config, 
                                  batch_size=1, 
                                  checkpoint_dir_or_path=os.path.expanduser('~/ECE324/cat-mel_2bar_big.tar'))
    
    def process(self, ns):
        print('Processing %s::%s (%f)' % (ns.id, ns.filename, ns.total_time))
        if ns.total_time > 60 * 5:
            print('Skipping notesequence with >5 minute duration')
            Metrics.counter('EncodeSong', 'skipped_long_song').inc()
            return
        
        Metrics.counter('EncodeSong', 'encoding_song').inc()

        chunk_length = 2
        melodies = song_utils.extract_melodies(ns)
        if not melodies:
            Metrics.counter('EncodeSong', 'extracted_no_melodies').inc()
            return
        Metrics.counter('EncodeSong', 'extracted_melody').inc(len(melodies))
        songs = [
            song_utils.Song(melody, self.model_config.data_converter,
                            chunk_length) for melody in melodies
        ]
        encoding_matrices = song_utils.encode_songs(self.model, songs)

        for matrix in encoding_matrices:
            assert matrix.shape[0] == 3 and matrix.shape[-1] == 512
            if matrix.shape[1] == 0:
                Metrics.counter('EncodeSong', 'skipped_matrix').inc()
                continue
            yield pickle.dumps(matrix)

Setup Apache Beam pipeline and process NS into LE with MusicVAE

In [3]:
# @title Apache Beam Pipeline
INPUT_NS_PATH = 'notesequences_mini.tfrecord' if MINI_DATASET else 'notesequences.tfrecord'
OUTPUT_EMBEDDING_PATH = 'embeddings_mini.tfrecord' if MINI_DATASET else 'embeddings.tfrecord'
with beam.Pipeline() as p:
    print("Starting Pipe")
    p |= 'tfrecord_list' >> beam.Create([os.path.expanduser('~/ECE324/' + INPUT_NS_PATH)])
    p |= 'read_tfrecord' >> beam.io.tfrecordio.ReadAllFromTFRecord(
        coder=beam.coders.ProtoCoder(note_seq.NoteSequence))
    p |= 'shuffle_input' >> beam.Reshuffle()
    p |= 'encode_song' >> beam.ParDo(Encode())
    p |= 'shuffle_output' >> beam.Reshuffle()
    p |= 'write' >> beam.io.WriteToTFRecord(os.path.expanduser( '~/ECE324/' + OUTPUT_EMBEDDING_PATH))

Starting Pipe


/home/david/.local/lib/python3.8/site-packages/magenta/models/music_vae/lstm_utils.py:94: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  tf.layers.dense(
/home/david/.local/lib/python3.8/site-packages/magenta/contrib/rnn.py:749: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._kernel = self.add_variable(
/home/david/.local/lib/python3.8/site-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._bias = self.add_variable(
/home/david/.local/lib/python3.8/site-packages/magenta/models/music_vae/base_model.py:195: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  mu = tf.layers.dense(
/home/david/.l

Processing /id/midi/0/e44e3411cdaf1059ed65c6a1b87fb61b29a27f8e::0d0bbede360f05c46e613c48945a2d81.mid (62.796875)
Processing /id/midi/0/05f56b2eed5b745dbb0820ff078447dda0738f93::01d32f61a42aa80d4ca13df400e00fa7.mid (153.970848)
Processing /id/midi/0/e62762c9f9cfbafdffef4b55e15eca55fd8a02bf::0fc67730a3f8208a4084477edb42cb98.mid (67.591545)
Processing /id/midi/0/1a70d0a60f51a9acf08af71dc60d02efb3488fe8::0be3fa76d37bd846b0058ec8be772c67.mid (28.805527)
Processing /id/midi/0/5b45a0c2dda9842046f3deeef6990220181de25e::02a8bb96d8be8bbfbd8fa5585b08120b.mid (24.930982)
Processing /id/midi/0/921a12647ce3c205df2e0a00a63d494b632c65aa::03cb6351c4c0a5d9291056b4af3b6d51.mid (232.102253)
Processing /id/midi/0/b396988794629619f88fe646a2f08c76fa502b6f::04d997858bec72efa933b9df71b2adc4.mid (78.566715)
Processing /id/midi/0/483aa1498471bd177b5d71f0e1e578b310232a06::0f966a97515bf45f378e6ef1e609b620.mid (142.559896)
Processing /id/midi/0/e623a1defd3d38058fc97e74703b14e81e5ddf92::0c7c6798082aeaa4a5b278c7e3cf0